In [43]:
import os
from bs4 import BeautifulSoup
import requests
import time
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [88]:
url = 'https://ycharts.com/companies/TSLA/revenues'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
response = requests.get(url, headers=headers)
if response.status_code != 200:
    raise Exception(f"Error al acceder a la URL: {url}")

contenido = response.content #scrapped content
soup = BeautifulSoup(contenido, 'html.  parser') # consultar para qué querria esto si con pandas lo puedo hacer (en tablas)

In [93]:
tables = soup.find_all('table', class_='table') #encuentro las tablas
for table in tables:
    # Encuentra las filas y celdas de la tabla
    rows = table.find_all('tr')
    for row in rows:
        cells = row.find_all('td')
        # Extraer datos de las celdas
        for cell in cells:
            data = cell.text.strip()
            print(data)

June 30, 2024
25.50B
March 31, 2024
21.30B
December 31, 2023
25.17B
September 30, 2023
23.35B
June 30, 2023
24.93B
March 31, 2023
23.33B
December 31, 2022
24.32B
September 30, 2022
21.45B
June 30, 2022
16.93B
March 31, 2022
18.76B
December 31, 2021
17.72B
September 30, 2021
13.76B
June 30, 2021
11.96B
March 31, 2021
10.39B
December 31, 2020
10.74B
September 30, 2020
8.771B
June 30, 2020
6.036B
March 31, 2020
5.985B
December 31, 2019
7.384B
September 30, 2019
6.303B
June 30, 2019
6.35B
March 31, 2019
4.541B
December 31, 2018
7.226B
September 30, 2018
6.824B
June 30, 2018
4.002B
March 31, 2018
3.409B
December 31, 2017
3.288B
September 30, 2017
2.985B
June 30, 2017
2.790B
March 31, 2017
2.696B
December 31, 2016
2.285B
September 30, 2016
2.298B
June 30, 2016
1.270B
March 31, 2016
1.147B
December 31, 2015
1.214B
September 30, 2015
936.79M
June 30, 2015
954.98M
March 31, 2015
939.88M
December 31, 2014
956.66M
September 30, 2014
851.80M
June 30, 2014
769.35M
March 31, 2014
620.54M
December 31

In [87]:
def extract_data_with_pandas(html_file):
    dfs = pd.read_html(html_file) # Leo todas las tablas del archivo HTML en una lista de DataFrames
    #print(dfs)
    result_df = pd.concat(dfs, ignore_index=True) # Concatena todos los DataFrames en uno solo
    return result_df

df = extract_data_with_pandas(contenido)

pd.set_option('display.max_rows', None)
print(df)

                  Date    Value                                0        1
0        June 30, 2024   25.50B                              NaN      NaN
1       March 31, 2024   21.30B                              NaN      NaN
2    December 31, 2023   25.17B                              NaN      NaN
3   September 30, 2023   23.35B                              NaN      NaN
4        June 30, 2023   24.93B                              NaN      NaN
5       March 31, 2023   23.33B                              NaN      NaN
6    December 31, 2022   24.32B                              NaN      NaN
7   September 30, 2022   21.45B                              NaN      NaN
8        June 30, 2022   16.93B                              NaN      NaN
9       March 31, 2022   18.76B                              NaN      NaN
10   December 31, 2021   17.72B                              NaN      NaN
11  September 30, 2021   13.76B                              NaN      NaN
12       June 30, 2021   11.96B       

In [79]:
tables = soup.find_all('table', class_='table') #encuentro las tablas

all_columns = []
data_list = []
for table in tables : 
    thead = table.find('thead')
    if thead: 
        headers = thead.find_all('th')
    else:
        headers = table.find('tbody').find_all('tr')[0].find_all('th')
    columns = [header.text.strip() for header in headers]
    all_columns.append(columns)
    rows = table.find('tbody').find_all('tr')
    for row in rows:
        columns = row.find_all('td')
        data = [column.text.strip() for column in columns]
        data_list.append(data)

print(all_columns)
print(data_list)

df = pd.DataFrame(columns=columns, data=data_list)
df


[['Date', 'Value'], ['Date', 'Value'], [], []]
[['June 30, 2024', '25.50B'], ['March 31, 2024', '21.30B'], ['December 31, 2023', '25.17B'], ['September 30, 2023', '23.35B'], ['June 30, 2023', '24.93B'], ['March 31, 2023', '23.33B'], ['December 31, 2022', '24.32B'], ['September 30, 2022', '21.45B'], ['June 30, 2022', '16.93B'], ['March 31, 2022', '18.76B'], ['December 31, 2021', '17.72B'], ['September 30, 2021', '13.76B'], ['June 30, 2021', '11.96B'], ['March 31, 2021', '10.39B'], ['December 31, 2020', '10.74B'], ['September 30, 2020', '8.771B'], ['June 30, 2020', '6.036B'], ['March 31, 2020', '5.985B'], ['December 31, 2019', '7.384B'], ['September 30, 2019', '6.303B'], ['June 30, 2019', '6.35B'], ['March 31, 2019', '4.541B'], ['December 31, 2018', '7.226B'], ['September 30, 2018', '6.824B'], ['June 30, 2018', '4.002B'], ['March 31, 2018', '3.409B'], ['December 31, 2017', '3.288B'], ['September 30, 2017', '2.985B'], ['June 30, 2017', '2.790B'], ['March 31, 2017', '2.696B'], ['December 3

,[[Normalized Earnings Yield]],[\n \n \n 1.835\n \n \n ]
0,"June 30, 2024",25.50B
1,"March 31, 2024",21.30B
2,"December 31, 2023",25.17B
3,"September 30, 2023",23.35B
4,"June 30, 2023",24.93B
...,...,...
59,Gross Profit Margin (Quarterly),17.95%
60,Profit Margin (Quarterly),5.80%
61,Earnings Yield,1.76%
62,Operating Earnings Yield,1.03%


In [ ]:
url_1 = 'https://www.promiedos.com.ar/primera'
response1 = requests.get(url_1)
if response1.status_code != 200:
    raise Exception(f"Error al acceder a la URL: {url}")
cont_promiedos = response1.content
soup_2 = BeautifulSoup(cont_promiedos, 'html.parser')

In [ ]:
print(soup_2)